In [ ]:
import pandas as pd
import numpy as np

#load nodes df
nodes_df = pd.read_csv('prepped_models/cifar10_testing/node_ranks.csv')

#make wide version
nodes_wide_df = nodes_df.pivot(index = 'node_num',columns='class', values='rank_score')

def get_col(node_num, df = nodes_df, idx = 'node_num', col = 'layer'):
    return df.loc[(df[idx] == node_num) & (df['class'] == df['class'].unique()[0]), col].item()

nodes_wide_df.reset_index(inplace=True)
nodes_wide_df['layer'] = nodes_wide_df['node_num'].apply(get_col)
nodes_wide_df = nodes_wide_df.rename(columns = {'class':'index'})


#list of layer nodes
layer_nodes = {}
for row in nodes_df[nodes_df['class'] == 'overall'].itertuples(): 
    if row.layer not in layer_nodes:
        layer_nodes[row.layer] = []
    layer_nodes[row.layer].append(row.node_num)

num_layers = max(layer_nodes.keys()) + 1
num_nodes = len(nodes_wide_df.index)

#list of classes
classes = list(nodes_df['class'].unique())
classes.remove('overall')
classes.insert(0,'overall')


nodes_wide_df.head(10)



In [311]:
edges_df = pd.read_csv('prepped_models/cifar10_testing/edge_ranks.csv')   #load edges

#make edges wide format df
edges_wide_df = edges_df.pivot(index = 'edge_num',columns='class', values='rank_score')
edges_wide_df.reset_index(inplace=True)
edges_wide_df['layer'] = edges_wide_df['edge_num'].apply(get_col, df=edges_df,idx='edge_num')
edges_wide_df['in_channel'] = edges_wide_df['edge_num'].apply(get_col, df=edges_df,idx='edge_num',col='in_channel')
edges_wide_df['out_channel'] = edges_wide_df['edge_num'].apply(get_col, df=edges_df,idx='edge_num',col='out_channel')

print(edges_df.head(4605))
edges_wide_df.head(165)
#edges_df.loc[(edges_df['rank_score'] > .05) & (edges_df['class'] == 'frog')]

num_edges = len(edges_wide_df.index) #number of total edges


      edge_num  layer  out_channel  in_channel  rank_score  class
0            0      0            0           0    0.017843   frog
1            1      0            0           1    0.016695   frog
2            2      0            0           2    0.093749   frog
3            3      0            1           0    0.084601   frog
4            4      0            1           1    0.043617   frog
...        ...    ...          ...         ...         ...    ...
4600         0      0            0           0    0.037768  horse
4601         1      0            0           1    0.021887  horse
4602         2      0            0           2    0.052484  horse
4603         3      0            1           0    0.042027  horse
4604         4      0            1           1    0.050973  horse

[4605 rows x 6 columns]


In [312]:
#misc formatting functions

def nodeid_2_perlayerid(nodeid):    #takes in node unique id outputs tuple of layer and within layer id
    if isinstance(nodeid,str):
        if not nodeid.isnumeric():
            layer = 'img'
            within_layer_id = imgnode_names.index(nodeid)
            return layer,within_layer_id
    nodeid = int(nodeid)
    layer = nodes_df[nodes_df['class']=='overall'][nodes_df['node_num'] == nodeid]['layer'].item()
    within_layer_id = nodes_df[nodes_df['class']=='overall'][nodes_df['node_num'] == nodeid]['node_num_by_layer'].item()
    return layer,within_layer_id

def layernum2name(layer,offset=1,title = 'layer'):
    return title+' '+str(layer+offset)


In [313]:
#generate mds projections of nodes layerwise, as determined by their per class rank scores

import numpy as np
from sklearn import manifold
from sklearn.metrics import euclidean_distances

def add_norm_col(df,classes=classes[1:]):
    norms = []
    norm = 0
    for index, row in df.iterrows():
        for label in classes:
            norm += row[label]**2
        norm = np.sqrt(norm)
        norms.append(norm)
    norms = np.array(norms)
    df['class_norm'] = norms

add_norm_col(nodes_wide_df)   
    
layer_similarities = {}
for layer in layer_nodes:
    layer_df = nodes_wide_df[nodes_wide_df['layer'] == layer]
    for label in classes:
        layer_df[label] = layer_df.apply(lambda row : row[label]/row['class_norm'], axis = 1)   
    layer_similarities[layer] = euclidean_distances(layer_df.iloc[:,1:-2])



layer_mds = {}
for layer in layer_similarities:
	print('layer: %s'%str(layer))
	mds = manifold.MDS(n_components=2, max_iter=3000, eps=1e-9, 
      random_state=2, dissimilarity="precomputed", n_jobs=1)
	pos = mds.fit(layer_similarities[layer]).embedding_
	layer_mds[layer] = pos

#print(layer_mds)





/Users/chrishamblin/miniconda3/envs/graph_viz/lib/python3.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/chrishamblin/miniconda3/envs/graph_viz/lib/python3.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/chrishamblin/miniconda3/envs/graph_viz/lib/python3.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

layer: 0
layer: 1
layer: 2
layer: 3


In [314]:
#generate node colors based on target class (nodes that aren't important should be faded)

target_class = 'airplane'

#Node Opacity
layer_colors = ['rgba(31,119,180,', 
                'rgba(255,127,14,',
                'rgba(44,160,44,', 
                'rgba(214,39,40,',
                'rgba(39, 208, 214,', 
                'rgba(242, 250, 17,',
                'rgba(196, 94, 255,',
                'rgba(193, 245, 5,',
                'rgba(245, 85, 5,',
                'rgba(5, 165, 245,',
                'rgba(245, 5, 105,',
                'rgba(218, 232, 23,',
                'rgba(148, 23, 232,',
                'rgba(23, 232, 166,',]


def node_color_scaling(x):
    return -(x-1)**4+1

def gen_node_colors(target_class):

    node_colors = []
    node_weights = []
    for layer in layer_nodes:
        node_colors.append([])
        node_weights.append([])
        for node in layer_nodes[layer]:
            node_weight = nodes_df[nodes_df['class']==target_class].iloc[node].rank_score
            node_weights[-1].append(node_weight)
            alpha = node_color_scaling(node_weight)
            node_colors[-1].append(layer_colors[layer%len(layer_colors)]+str(round(alpha,3))+')')
            
    return node_colors

node_colors,node_weights = gen_node_colors(target_class)     #list of lists


In [315]:
#Node positions
#def gen_node_positions()
layer_distance = 1   # distance in X direction each layer is separated by
node_positions = []
layer_offset = 0
for layer in layer_mds:
    node_positions.append({})
    node_positions[-1]['X'] = [] 
    node_positions[-1]['Y'] = [] 
    node_positions[-1]['Z'] = []  
    for i in range(len(layer_mds[layer])): 
        node_positions[-1]['Y'].append(layer_mds[layer][i][0])
        node_positions[-1]['Z'].append(layer_mds[layer][i][1])
        node_positions[-1]['X'].append(layer_offset)
    layer_offset+=1*layer_distance

#print(node_positions[0])

In [316]:
#image nodes (one for each channel of input image)

num_img_chan = len(edges_df.loc[edges_df['layer'] == 0]['in_channel'].unique()) #number of channels in input image

def gen_imgnode_graphdata(num_chan = num_img_chan):     #returns positions, colors and names for imgnode graph points
    if num_chan == 1: #return a centered position, grey square, with 'gs' label
        return {'X':[-1*layer_distance],'Y':[0],'Z':[0]}, ['rgba(170,170,170,.7)'], ['gs']
    if num_chan == 3:
        colors = ['rgba(255,0,0,.7)','rgba(0,255,0,.7)','rgba(0,0,255,.7)']
        names = ['r','g','b']
    else:
        #colors
        other_colors = ['rgba(255,0,0,.7)','rgba(0,255,0,.7)','rgba(0,0,255,.7)',
                        'rgba(255,150,0,.7)','rgba(0,255,150,.7)','rgba(150,0,255,.7)',
                        'rgba(255,0,150,.7)','rgba(150,255,0,.7)','rgba(0,150,255,.7)']
        colors = []
        for i in num_chan:
            colors.append(i%len(other_colors)) 
        #names
        names = []
        for i in range(num_chan):
            names.append('img_'+str(i))   
            
    positions = {'X':[],'Y':[],'Z':[]}     #else return points evenly spaced around a unit circle
    a = 2*np.pi/num_chan          #angle to rotate each point
    for p in range(num_chan):
        positions['X'].append(-1*layer_distance)
        positions['Y'].append(round(np.sin(a*p)/5,2))
        positions['Z'].append(round(np.cos(a*p)/5,2)) 
    
    return positions, colors, names
    
# def gen_imgnode_positions(num_chan = num_img_chan):

#     if num_chan == 1: #return a centered position
#         return {'X':[-1],'Y':[0],'Z':[0]}
    
#     positions = {'X':[],'Y':[],'Z':[]}     #else return points evenly spaced around a unit circle
#     a = 2*np.pi/num_chan          #angle to rotate each point
#     for p in range(num_chan):
#         positions['X'].append(-1)
#         positions['Y'].append(np.sin(a*p))
#         positions['Z'].append(np.cos(a*p))
#     return positions

# def gen_imgnode_colors(num_chan = num_img_chan):
#     if num_chan == 1:
#         return ['rgba(170,170,170,.9)']   #grey
#     elif num_chan == 3:
#         return ['rgba(255,0,0,.9)','rgba(0,255,0,.9)','rgba(0,0,255,.9)']             #rgb

#     else:
#         other_colors = ['rgba(255,0,0,.9)','rgba(0,255,0,.9)','rgba(0,0,255,.9)',
#                         'rgba(255,150,0,.9)','rgba(0,255,150,.9)','rgba(150,0,255,.9)',
#                         'rgba(255,0,150,.9)','rgba(150,255,0,.9)','rgba(0,150,255,.9)']
#         colors = []
#         for i in num_chan:
#             colors.append(i%len(other_colors))
#         return colors
    
# def gen_imgnode_names(num_chan = num_img_chan):
#     if num_chan == 1:
#         return ['gs'] #grayscale
#     elif num_chan == 3:
#         return ['r','g','b']
#     else:
#         names = []
#         for i in range(num_chan):
#             names.append('img_'+str(i))
#         return names

imgnode_positions,imgnode_colors,imgnode_names = gen_imgnode_graphdata()

print(imgnode_positions)

{'X': [-1, -1, -1], 'Y': [0.0, 0.17, -0.17], 'Z': [0.2, -0.1, -0.1]}


In [317]:
#Edge selection

def edge_width_scaling(x):
    return max(.4,(x*10)**1.7)

def edge_color_scaling(x):
    return max(.7,-(x-1)**4+1)


def get_thresholded_edges(threshold,df=edges_df,target_class=target_class):          #just get those edges that pass the threshold criteria for the target class
    if len(threshold) != 2:
        raise Exception('length of threshold needs to be two ([lower, higher])')
    return edges_df.loc[(edges_df['rank_score'] >= threshold[0]) & (edges_df['rank_score'] <= threshold[1]) & (edges_df['class'] == target_class)]
    #return edges_df.loc[(threshold[0] <= edges_df['rank_score'] <= threshold[1]) & (edges_df['class'] == target_class)]
    #return edges_df.loc[(edges_df['rank_score'] >= threshold[0]) & (edges_df['class'] == target_class)]


edge_threshold = [.1,1]
edges_select_df = get_thresholded_edges(edge_threshold)



def get_max_edge_widths(edge_widths):
    maxes = []
    for layer in range(len(edge_widths)):
        maxes.append(edge_widths[layer].index(max(edge_widths[layer])))
    return maxes

def gen_edge_graphdata(df = edges_select_df, node_positions = node_positions, num_hoverpoints=15,target_class=target_class):
    edge_positions = []
    colors = []
    widths = []
    weights = []
    names = []
    max_weight = 0
    for row in df.itertuples():
        if row.layer > len(edge_positions): # we skipped a layer, its got no edges in threshold, so lets add empty lists
            edge_positions.append({'X':[],'Y':[],'Z':[]})
            colors.append([])
            widths.append([])
            weights.append([])
            names.append([])  
        if row.layer == len(edge_positions):
            edge_positions.append({'X':[],'Y':[],'Z':[]})
            colors.append([])
            widths.append([])
            weights.append([])
            names.append([])        
        #position
        for dim in ['X','Y','Z']:
            end_pos = node_positions[row.layer][dim][row.out_channel]
            if row.layer != 0:
                start_pos = node_positions[row.layer-1][dim][row.in_channel]
            else:
                start_pos = imgnode_positions[dim][row.in_channel]
            
            step = (end_pos-start_pos)/(num_hoverpoints+1)
            points = [start_pos]
            for i in range(1,num_hoverpoints+1):
                points.append(start_pos+i*step)
            points.append(end_pos)
            edge_positions[row.layer][dim].append(points)
        #color
        alpha = edge_color_scaling(row.rank_score)
        colors[row.layer].append(layer_colors[row.layer%len(layer_colors)]+str(round(alpha,3))+')')
        #width
        widths[row.layer].append(edge_width_scaling(row.rank_score))
        #weight
        weights[row.layer].append(row.rank_score)
        max_weight = max(max_weight, row.rank_score)
        #names
        out_node = layer_nodes[row.layer][row.out_channel]
        if row.layer != 0:
            in_node = layer_nodes[row.layer-1][row.in_channel]
        else:
            in_node = imgnode_names[row.in_channel]
        names[row.layer].append(str(in_node)+'-'+str(out_node))
    max_width_indices = get_max_edge_widths(widths)
    return edge_positions, colors,widths,weights,names,max_width_indices, max_weight




def get_edge_from_curvenumber(curvenum,edge_names, num_layers= num_layers):
    edgenum = curvenum-(1+num_layers)
    curve=0
    for layer in range(len(edge_names)):
        for i in range(len(edge_names[layer])):
            if curve==edgenum:
                return layer, i, edge_names[layer][i]
            curve+=1
    return None,None,None
    
edge_positions, edge_colors, edge_widths, edge_weights, edge_names, max_edge_width_indices,max_edge_weight = gen_edge_graphdata()
print(len(edge_names[0]))


27


In [318]:
#Format Node Feature Maps

# import pickle

# activations = pickle.load(open('activations/cifar_prunned_.816_activations.pkl','rb'))
  
    
# node_ids = []
# for layer in layer_nodes:
#     for i in range(len(layer_nodes[layer])):
#         node_ids.append(str(layer+1)+'_'+str(i))
    
# print(activations['airplane']['0001.png'][0].shape)

import torch
activations = torch.load('prepped_models/cifar10_testing/input_img_activations.pt')

print(activations['edges'][0].shape)
print(activations['nodes'][0].shape)

(100, 45, 3, 32, 32)
(100, 45, 32, 32)


In [319]:
#Format Edge Kernels

# kernels = pickle.load(open('kernels/cifar_prunned_.816_kernels.pkl','rb'))
# print(kernels[0].shape)

kernels = torch.load('prepped_models/cifar10_testing/kernels.pt')

print(kernels[0].shape)

#Function for taking a string of form 'node1-node2' and outputting edge info
def nodestring_2_edge_info(nodestring):
    from_node = nodestring.split('-')[0]
    to_node = nodestring.split('-')[1]
    from_layer,from_within_id = nodeid_2_perlayerid(from_node)
    to_layer,to_within_id = nodeid_2_perlayerid(to_node)
    kernel = kernels[to_layer][to_within_id][from_within_id]
    return np.flip(kernel,0)


print(nodestring_2_edge_info('b-0'))
print(np.flip(kernels[0][0][2],0))

(45, 3, 3, 3)
[[-0.01040168  0.07258165 -0.02730235]
 [-0.09874548  0.15988253  0.16769743]
 [-0.11188427  0.21347913 -0.24422328]]
[[-0.01040168  0.07258165 -0.02730235]
 [-0.09874548  0.15988253  0.16769743]
 [-0.11188427  0.21347913 -0.24422328]]


/Users/chrishamblin/miniconda3/envs/graph_viz/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/Users/chrishamblin/miniconda3/envs/graph_viz/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [320]:
## adding images

import os

input_image_directory = 'input_images_testing/'
list_of_input_images = os.listdir(input_image_directory)
list_of_input_images.sort()

from PIL import Image

def rgb2hex(r, g, b):
    return '#{:02x}{:02x}{:02x}'.format(r, g, b)


def image2heatmap(image_path,resize = False,size = (32,32)):          #displays image as a plotly heatmap object, with colors preserved 
    img = Image.open(image_path)
    if resize:
        img = img.resize(size,resample=Image.NEAREST)
    pixels = img.convert('RGBA').load() #rgba values
    width, height = img.size   #width and height of image
    num_pixels = width*height
    step = 1/num_pixels
    
    colorscale = []           
    z = []
    
    i = 0
    for y in range(height):
        z.append([])
        for x in range(width):
            z[-1].append(step*(i+.5))
            r, g, b, a = pixels[x, y]
            #colorscale.append([step*i,'rgba(%s,%s,%s,%s)'%(r,b,g,a)])
            #colorscale.append([step*(i+1),'rgba(%s,%s,%s,%s)'%(r,b,g,a)])
            colorscale.append([step*i,rgb2hex(r, g, b)])
            colorscale.append([step*(i+1),rgb2hex(r, g, b)])
            i+=1     
    heatmap = go.Heatmap(z=np.flip(np.array(z),0), 
                     colorscale = colorscale, 
                     showscale = False)
    fig = go.Figure(data=[heatmap])

    fig.update_layout(width=500, height=500)
    return fig




In [321]:
#hidden state, stores python values within the html itself
state = {'edge_positions':edge_positions,'edge_colors': edge_colors, 'edge_widths':edge_widths,'edge_names':edge_names,
         'edge_threshold':edge_threshold,'edge_weights':edge_weights,'max_edge_width_indices':max_edge_width_indices,
         'max_edge_weight':max_edge_weight,'imgnode_positions':imgnode_positions,'imgnode_colors':imgnode_colors,'imgnode_names':imgnode_names,
         'node_positions':node_positions,'node_colors':node_colors,'node_weights':node_weights,'layer_distance':layer_distance,'target_class':target_class,
         'node_select_history':['0'],'edge_select_history':['r-0']}


In [322]:
#import chart_studio.plotly as py
import plotly.offline as py    #added
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)   #added

from copy import deepcopy

# right now this is called everytime graph is updated, might be more efficient to store combined_traces somehow,
# and only update whats changed
def gen_networkgraph_traces(state):
    #add imgnodes
    colors = deepcopy(state['imgnode_colors'])
    if not state['node_select_history'][-1].isnumeric():
        colors[state['imgnode_names'].index(state['node_select_history'][-1])] = 'rgba(0,0,0,1)'
    imgnode_trace=go.Scatter3d(x=state['imgnode_positions']['X'],
               y=state['imgnode_positions']['Y'],
               z=state['imgnode_positions']['Z'],
               mode='markers',
               name='image channels',
               marker=dict(symbol='square',
                             size=8,
                             opacity=.99,
                             color=colors,
                             #colorscale='Viridis',
                             line=dict(color='rgb(50,50,50)', width=.5)
                             ),
               text=state['imgnode_names'],
               hoverinfo='text'
               )

    imgnode_traces = [imgnode_trace]


    node_traces = []
    select_layer,select_position = None,None
    if state['node_select_history'][-1].isnumeric():
        select_layer,select_position = nodeid_2_perlayerid(state['node_select_history'][-1])
    for layer in layer_nodes:
        #add nodes
        colors = deepcopy(state['node_colors'][layer])
        if layer == select_layer:
            colors[select_position] = 'rgba(0,0,0,1)'
        node_trace=go.Scatter3d(x=state['node_positions'][layer]['X'],
                   y=state['node_positions'][layer]['Y'],
                   z=state['node_positions'][layer]['Z'],
                   mode='markers',
                   name=layernum2name(layer,title = 'nodes'),
                   marker=dict(symbol='circle',
                                 size=6,
                                 opacity=.99,
                                 color=colors,
                                 #colorscale='Viridis',
                                 line=dict(color='rgb(50,50,50)', width=.5)
                                 ),
                   text=layer_nodes[layer],
                   hoverinfo='text'
                   )

        node_traces.append(node_trace)


    edge_traces = []
    for layer in range(len(state['edge_positions'])):  
        legendgroup = layernum2name(layer ,title = 'edges')
        for edge_num in range(len(state['edge_positions'][layer]['X'])):  
        #add edges      
            color = deepcopy(state['edge_colors'][layer][edge_num])
            if state['edge_names'][layer][edge_num] == state['edge_select_history'][-1]:
                color = 'rgba(0,0,0,1)'
            showlegend = False
            if state['max_edge_width_indices'][layer] == edge_num:
                showlegend = True
            edge_trace=go.Scatter3d(x=state['edge_positions'][layer]['X'][edge_num],
                                    y=state['edge_positions'][layer]['Y'][edge_num],
                                    z=state['edge_positions'][layer]['Z'][edge_num],
                                    legendgroup=legendgroup,
                                    showlegend=showlegend,
                                    name=layernum2name(layer ,title = 'edges'),
                                    mode='lines',
                                    #line=dict(color=edge_colors_dict[layer], width=1.5),
                                    line=dict(color=color, width=state['edge_widths'][layer][edge_num]),
                                    text = state['edge_names'][layer][edge_num],
                                    hoverinfo='text'
                                    )
            edge_traces.append(edge_trace)


    combined_traces = imgnode_traces+node_traces+edge_traces
    return combined_traces

combined_traces = gen_networkgraph_traces(state)


#layout
axis=dict(showbackground=False,
          showspikes=False,
          showline=False,
          zeroline=False,
          showgrid=False,
          showticklabels=False,
          #range=[0,0],
          title=''
          )

graph_layout = go.Layout(
         #title="%s through Prunned Cifar10 CNN"%target_class,
         #title = target_class,
         #width=1000,
         clickmode = 'event+select',
         transition = {'duration': 500},
         height=600,
         #showlegend=False,
         margin = dict(l=20, r=20, t=20, b=20),
         scene=dict(
             xaxis=dict(axis),
             yaxis=dict(axis),
             zaxis=dict(axis),
             aspectmode ="manual", 
             aspectratio = dict(x=1, y=0.5, z=0.5) #adjusting this stretches the network layer-to-layer
         ),
         uirevision =  True   
         #hovermode='closest',
   )


network_graph_fig=go.Figure(data=combined_traces, layout=graph_layout)

#state['combined_traces']=combined_traces


/Users/chrishamblin/miniconda3/envs/graph_viz/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/Users/chrishamblin/miniconda3/envs/graph_viz/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [330]:
state['max_edge_width']

12.423941493147577

In [325]:
import dash
import dash_core_components as dcc
import dash_html_components as html
#import utils.dash_reusable_components as drc
import flask
import os

import json

from dash.dependencies import Input, Output, State


#external_stylesheets = ['https://codepen.io/amyoshino/pen/jzXypZ.css']
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(external_stylesheets = external_stylesheets)


styles = {
    'pre': {
        'border': 'thin lightgrey solid',
        'overflowX': 'scroll'
    }
}


app.layout = html.Div(
        [html.Div(         #Left side control panel
            children = [
             html.Label('Weighting Category'),
             dcc.Dropdown(
                id='weight-category',
                options=[{'label': i, 'value': i} for i in classes],
                value=target_class
                ),
             html.Br(),
             html.Label('Weighting Criterion'),
             dcc.Dropdown(
                id='weight-criterion',
                options=[
                    {'label': 'Activations*Grads', 'value': 'actgrads'},
                    {'label': 'Activations', 'value': 'acts'}
                ],
                value='actgrads'
                ),
             html.Br(),   
             html.Label('Layer Projection'),
             dcc.Dropdown(
                id = 'layer-projection',
                options=[
                    {'label': 'MDS', 'value': 'MDS'},
                    {'label': 'Grid', 'value': 'grid'},
                    #{'label': 'SOM', 'value': 'SOM'}
                ],
                value='MDS'
                ),

            html.Br(),
            html.Label('Edge Thresholds'),
                dcc.RangeSlider(
                    id='edge-thresh-slider',
                    min=0,
                    max=np.ceil(state['max_edge_width']*10)/10,
                    step=0.001,
                    marks={i/10: str(i/10) for i in range(0,int(np.ceil(state['max_edge_width']*10))+1,int(round(np.ceil(state['max_edge_width']*10)/5)))},
                    value=[.1,np.ceil(state['max_edge_width']*10)/10],
                ),
                
            ], className="two columns"
        ),

        html.Div(
            children = [
                
            html.Div([
                dcc.Graph(
                    id='network-graph',
                    figure=network_graph_fig
                )
            ], className= 'row'
            ),
                
            html.Div([
                html.Div([
                html.Label('Input Image'),
                dcc.Dropdown(
                    id='input-image-dropdown',
                    options=[{'label': i, 'value': i} for i in list_of_input_images],
                    value=list_of_input_images[0]
                ),

                dcc.Graph(
                    id='img-actmap-graph',
                    figure=image2heatmap(input_image_directory+list_of_input_images[0]),
                    config={
                            'displayModeBar': False
                            }
                )
                ], className = "three columns"),
                
                html.Div([
                html.Label('Node'),
                dcc.Dropdown(
                    id='node-actmap-dropdown',
                    options=[{'label': str(i), 'value': i} for i in range(num_nodes)],
                    value=0
                ),
                dcc.Graph(
                    id='node-actmap-graph',
                    figure=go.Figure(data=go.Heatmap(
                                        z = np.flip(activations['nodes'][0][0][0],0)),
                                        layout=dict(
                                            height=500,
                                            width=500)
                                    ),
                    config={
                            'displayModeBar': False
                            }
                )
                ], className = "three columns"),
                
                html.Div([
                html.Label('Edge'),    
                dcc.Input(
                    id='edge-kernel-input',value='0-%s'%str(layer_nodes[1][0]), type='text'),
                html.Button(id='edge-kernel-button',n_clicks=0, children='Submit'),
                dcc.Graph(
                    id='edge-kernel-graph',
                    figure=go.Figure(data=go.Heatmap(
                                        z = nodestring_2_edge_info('0-%s'%str(layer_nodes[1][0]))
                                        ),
                                        layout=dict(
                                            height=500,
                                            width=500)
                            ),
                    config={
                            'displayModeBar': False
                            }
                )
                ], className = "three columns")
                
                
             ], className= 'row'
             ),
                
                
            html.Div([
                html.Div([
                    dcc.Markdown("""
                        **Hover Data**

                        Mouse over values in the graph.
                    """),
                    html.Pre(id='hover-data', style=styles['pre'])
                ], className='two columns'),

                html.Div([
                    dcc.Markdown("""
                        **Click Data**

                        Click on points in the graph.
                    """),
                    html.Pre(id='click-data', style=styles['pre']),
                ], className='two columns'),

                html.Div([
                    dcc.Markdown("""
                        **Selection Data**

                        Choose the lasso or rectangle tool in the graph's menu
                        bar and then select points in the graph.

                        Note that if `layout.clickmode = 'event+select'`, selection data also 
                        accumulates (or un-accumulates) selected data if you hold down the shift
                        button while clicking.
                    """),
                    html.Pre(id='selected-data', style=styles['pre']),
                ], className='two columns'),

#                 html.Div([
#                     dcc.Markdown("""
#                         **Zoom and Relayout Data**

#                         Click and drag on the graph to zoom or click on the zoom
#                         buttons in the graph's menu bar.
#                         Clicking on legend items will also fire
#                         this event.
#                     """),
#                     html.Pre(id='relayout-data', style=styles['pre']),
#                 ], className='two columns')
                
                html.Div([
                    dcc.Markdown("""
                        **Figure Data**

                        Figure json info.
                    """),
                    html.Pre(id='figure-data', style=styles['pre']),
                ], className='four columns')
                
            ], className= 'row'
            )
        ], className="ten columns"
        ),
    #hidden divs for storing intermediate values     
    # The memory store reverts to the default on every page refresh
    dcc.Store(id='memory'),
    # The local store will take the initial data
    # only the first time the page is loaded
    # and keep it until it is cleared.
    dcc.Store(id='local', storage_type='local'),
    # Same as the local store but will lose the data
    # when the browser/tab closes.
    dcc.Store(id='session', storage_type='session',data=state)
    ]
)



####Call Back Functions


# @app.callback(
#     [Output('network-graph', 'figure'),
#      Output('previous-click','children')],
#     [Input('network-graph', 'clickData'),
#      Input('previous-click','children')])
# def highlight_on_click(clickData,state,previous_click =False):
#     #if clickData['points'][0]['curveNumber'] == None:
#     if not clickData:
#         raise Exception('no point clicked yet') 
#     #recolor previous click 
#     if previous_click:
#         if previous_click == 0:
#             state['combined_traces'][0]['marker']['color'] = state['imgnode_colors']
#         elif previous_click < num_layers+1:
#             state['combined_traces'][previous_click]['marker']['color'] = state['node_colors'][previous_click-1]
#         else:
#             layer,position = get_edge_layer_from_curvenumber(previous_click,state['edge_names'])
#             state['combined_traces'][previous_click]['line']['color'] == state['edge_colors'][layer][position]
#     #blackout click
#     trace_num = int(clickData['points'][0]['curveNumber'])
#     if trace_num == 0:
#         new_colors = state['imgnode_colors']
#         new_colors[clickData['points'][0]['pointNumber']] = 'rgba(0,0,0,1)'        
#     elif trace_num < num_layers +1:   #highlight point
#         new_colors = list(state['node_colors'][trace_num-1])
#         new_colors[clickData['points'][0]['pointNumber']] = 'rgba(0,0,0,1)'
#         state['combined_traces'][trace_num]['marker']['color'] = new_colors
#     else: #highlight edge
#         state['combined_traces'][trace_num]['line']['color'] = 'rgba(0,0,0,1)'
#     return 
  
#     layout = graph_layout
#     layout['uirevision'] = True
#     previous_click = {'curveNumber':clickData['points'][0]['curveNumber'],'pointNumber':clickData['points'][0]['pointNumber']}
#     return {'data': combined_traces,'layout': layout},json.dumps(previous_click)



@app.callback(
    Output('session', 'data'),
    [Input('weight-category', 'value'),
     Input('network-graph', 'clickData'),
     Input('edge-thresh-slider','value')],
    [State('session', 'data')])
def update_store(target_class,clickData,edge_threshold,state):
    print('UPDATING STOOOOOOOORERE\nUPDATING STOOOOOOOORERE\nUPDATING STOOOOOOOORERE\n')
    ctx = dash.callback_context
    if not ctx.triggered:
        raise Exception('no figure updates yet')
    else:
        trigger = ctx.triggered[0]['prop_id']
        
    if trigger == 'weight-category.value':
        state['node_colors'], state['node_weights'] = gen_node_colors(target_class)
        print('changed target class to %s'%target_class)
        edges_select_df = get_thresholded_edges(threshold=edge_threshold,target_class=target_class)
        state['edge_positions'], state['edge_colors'], state['edge_widths'],state['edge_weights'], state['edge_names'],state['max_edge_width_indices'],state['max_edge_weight'] = gen_edge_graphdata(df = edges_select_df,node_positions = state['node_positions'],target_class=target_class)
    elif trigger == 'network-graph.clickData':
        if clickData is not None:
            if clickData['points'][0]['curveNumber'] > num_layers: #edge curve
                print('changing clicked edge')
                select_edge_layer, select_edge_position,edge_name = get_edge_from_curvenumber(clickData['points'][0]['curveNumber'],state['edge_names'])
                state['edge_select_history'].append(edge_name)
            elif clickData['points'][0]['curveNumber'] == 0:  #imgnode
                print('changing clicked node')
                state['node_select_history'].append(state['imgnode_names'][clickData['points'][0]['pointNumber']])
            else:
                print('changing clicked node')
                state['node_select_history'].append(layer_nodes[clickData['points'][0]['curveNumber']-1][clickData['points'][0]['pointNumber']])
            if len(state['node_select_history']) > 100:
                del state['node_select_history'][0]
            if len(state['edge_select_history']) > 100:
                del state['edge_select_history'][0]
    elif trigger == 'edge-thresh-slider.value':
        print('changing edge thresholds to %s - %s'%(edge_threshold[0],edge_threshold[1]))
        edges_select_df = get_thresholded_edges(threshold=edge_threshold,target_class=target_class)
        print('found %s edges'%len(edges_select_df))
        state['edge_positions'], state['edge_colors'], state['edge_widths'], state['edge_weights'], state['edge_names'],state['max_edge_width_indices'], state['max_edge_weight'] = gen_edge_graphdata(df = edges_select_df,node_positions = state['node_positions'],target_class=target_class)
        print(len(state['edge_names'][1]))
    else:
        raise Exception('unknown trigger: %s'%trigger)    
    return state



@app.callback(
    Output('network-graph', 'figure'),
    [Input('session', 'data')])
def update_figure(state):
    print('UPDATING FIGUUUUURE\nUPDATING FIGUUUUURE\nUPDATING FIGUUUUURE\n')
    combined_traces = gen_networkgraph_traces(state)    
    layout = graph_layout
    layout['uirevision'] = True
    return {'data': combined_traces,
            'layout': layout}

# @app.callback(
#     Output('network-graph', 'figure'),
#     [Input('weight-category', 'value'),
#      Input('network-graph', 'clickData'),
#      Input('lower-thresh-slider','value')],
#     [State('session', 'data')])
# def update_figure(target_class,clickData,edge_thresh,state):
#     ctx = dash.callback_context
#     if not ctx.triggered:
#         raise Exception('no figure updates yet')
#     else:
#         trigger = ctx.triggered[0]['prop_id']
        
#     if trigger == 'weight-category.value':
#         combined_traces = gen_networkgraph_traces()
#     elif trigger == 'network-graph.clickData':
#         previous_click = False
#         if len(state['click_history']['curveNumber']) > 1:
#             previous_click = state['click_history']['curveNumber'][-2]   #second to last click
#         highlight_on_click(clickData,state,previous_click=previous_click)
#     elif trigger == 'lower-thresh-slider.value':
#         combined_traces = gen_networkgraph_traces()
#     else:
#         raise Exception('unknown trigger: %s'%trigger)
        
#     layout = graph_layout
#     layout['uirevision'] = True
#     return {'data': combined_traces,
#             'layout': layout}

# @app.callback(
#     Output('node-actmap-dropdown', 'value'),
#     [Input('network-graph', 'clickData')])
# def switch_node_actmap_click(clickData):
#     if clickData['points'][0]['curveNumber'] == None:
#         raise Exception('no point clicked yet') 
#     if int(clickData['points'][0]['curveNumber']) >= num_layers:
#         raise Exception('Do nothing, they clicked an edge')
#     return int(clickData['points'][0]['text'])


         
# #cant currently click edges
# # @app.callback(
# #     Output('edge-image-dropdown', 'value'),
# #     [Input('network-graph', 'clickData')])
# # def switch_edge_image_click(clickData):
# #     if int(clickData['points'][0]['curveNumber']) < num_layers:
# #         raise Exception('Do nothing, they clicked a node')
# #     return list_of_edge_images[int(clickData['points'][0]['pointNumber'])]



# #Node activation map
# @app.callback(
#     Output('node-actmap-graph', 'figure'),
#     [Input('node-actmap-dropdown', 'value'),
#      Input('input-image-dropdown', 'value')])
# def update_node_actmap(nodeid,image_name):
#     layer, within_id = nodeid_2_perlayerid(nodeid)
#     if layer == 'img':
#         pass
#         #code for returning color channel as activation map
        
#     return go.Figure(data=go.Heatmap( z = np.flip(activations[image_name.split('_')[0]][image_name.split('_')[1]][layer][within_id],0)),
#                      layout=dict(height=500,
#                                  width=500,
#                                  uirevision=True)) 
# #     return {'data':go.Heatmap(
# #                               z = activations[image_name.split('_')[0]][image_name.split('_')[1]][layer][within_id]),
# #             'layout':dict(height=500,width=500)}
 
    
      

# @app.callback(
#     Output('edge-kernel-graph', 'figure'),
#     [Input('edge-kernel-button','n_clicks')],
#     [State('edge-kernel-input', 'value')])
# def update_edge_kernelmap(n_clicks,nodestring):
#     return go.Figure(data=go.Heatmap(z = nodestring_2_edge_info(nodestring)),
#                      layout=dict(height=500,
#                                  width=500,
#                                  uirevision=True)) 
                

# #Input Images
# @app.callback(
#     Output('input-image', 'src'),
#     [Input('input-image-dropdown', 'value')])
# def update_input_image_src(value):
#     return static_input_image_route + value

# @app.server.route('{}<image_path>.png'.format(static_input_image_route))
# def serve_input_image(image_path):
#     image_name = '{}.png'.format(image_path)
#     if image_name not in list_of_input_images:
#         raise Exception('"{}" is excluded from the allowed static files'.format(image_path))
#     return flask.send_from_directory(input_image_directory, image_name)




# #JSON INFO

@app.callback(
    Output('hover-data', 'children'),
    [Input('network-graph', 'hoverData')])
def display_hover_data(hoverData):
    return json.dumps(hoverData, indent=2)




@app.callback(
    Output('click-data', 'children'),
    [Input('network-graph', 'clickData')])
def display_click_data(clickData):
    return json.dumps(clickData, indent=2)


@app.callback(
    Output('selected-data', 'children'),
    [Input('network-graph', 'selectedData')])
def display_selected_data(selectedData):
    return json.dumps(selectedData, indent=2)






@app.callback(
    Output('figure-data', 'children'),
    [Input('weight-category', 'value'),
     Input('network-graph', 'clickData'),
     Input('edge-thresh-slider','value'),
     Input('session','data')])
def display_trigger(target_class,clickData,edge_thresh,state):
    ctx = dash.callback_context
    if not ctx.triggered:
        raise Exception('no figure updates yet')
    else:
        trigger = ctx.triggered[0]['prop_id'].split('.')[0]
    ctx_msg = json.dumps({
        'states': ctx.states,
        'triggered': ctx.triggered,
        'inputs': ctx.inputs,
        'full_state':state
    }, indent=2)
    return ctx_msg
    
    


       
    
#     node_colors = gen_node_colors(target_class)
#     Edges,edge_weights = gen_edge_subset_and_weights(target_class,edge_threshold=edge_thresh)
#     edge_positions = gen_edge_positions(Edges)
#     click_layer = int(clickData['points'][0]['curveNumber'])
#     for layer in node_colors:
#         if layer == click_layer:
#             new_colors = list(node_colors[click_layer])
#             new_colors[clickData['points'][0]['pointNumber']] = 'rgba(0,0,0,1)'
#             combined_traces[layer]['marker']['color'] = new_colors
#         else:
#             combined_traces[layer]['marker']['color'] = node_colors[layer]

#     for layer in edge_positions:
#         combined_traces[layer-1+num_layers] = go.Scatter3d(x=edge_positions[layer]['X'],
#                                 y=edge_positions[layer]['Y'],
#                                 z=edge_positions[layer]['Z'],
#                                 name=layernum2name(layer ,title = 'edges'),
#                                 mode='lines',
#                                 #line=dict(color=edge_colors_dict[layer], width=1.5),
#                                 line=dict(color='rgb(100,100,100)', width=1.5),
#                                 text = list(range(len(Edges[layer])))
#                                 #hoverinfo='text'
#                                 )
   
#     layout = graph_layout
#     layout['uirevision'] = True
#     return {'data': combined_traces,
#             'layout': layout}


# @app.callback(
#     Output('network-graph', 'figure'),
#     [Input('weight-category', 'value'),
#      Input('network-graph', 'clickData'),
#      Input('lower-thresh-slider','value')])
# def update_figure(target_class,clickData,edge_thresh):
#     node_colors = gen_node_colors(target_class)
#     Edges,edge_weights = gen_edge_subset_and_weights(target_class,edge_threshold=edge_thresh)
#     edge_positions = gen_edge_positions(Edges)
#     click_layer = int(clickData['points'][0]['curveNumber'])
#     for layer in node_colors:
#         if layer == click_layer:
#             new_colors = list(node_colors[click_layer])
#             new_colors[clickData['points'][0]['pointNumber']] = 'rgba(0,0,0,1)'
#             combined_traces[layer]['marker']['color'] = new_colors
#         else:
#             combined_traces[layer]['marker']['color'] = node_colors[layer]

#     for layer in edge_positions:
#         combined_traces[layer-1+num_layers] = go.Scatter3d(x=edge_positions[layer]['X'],
#                                 y=edge_positions[layer]['Y'],
#                                 z=edge_positions[layer]['Z'],
#                                 name=layernum2name(layer ,title = 'edges'),
#                                 mode='lines',
#                                 #line=dict(color=edge_colors_dict[layer], width=1.5),
#                                 line=dict(color='rgb(100,100,100)', width=1.5),
#                                 text = list(range(len(Edges[layer])))
#                                 #hoverinfo='text'
#                                 )
   
#     layout = graph_layout
#     layout['uirevision'] = True
#     return {'data': combined_traces,
#             'layout': layout}
















# @app.callback(
#     Output('relayout-data', 'children'),
#     [Input('network-graph', 'relayoutData')])
# def display_relayout_data(relayoutData):
#     return json.dumps(relayoutData, indent=2)





# @app.callback(
#     Output('figure-data', 'children'),
#     [Input('network-graph', 'figure')])
# def display_figure_data(figure):
#     return json.dumps(figure, indent=2)






# Add a static image route that serves images from desktop
# Be *very* careful here - you don't want to serve arbitrary files
# from your computer or server

# @app.callback(Output('my-div', 'children'),
#                      [Input('my-input', 'value')],
#                      [State('my-div', 'children')])
# def update_div(value, existing_state):
#     if some_condition:
#          return existing_state


#py.iplot(fig, filename='small_net')



#             html.Label('Multi-Select Dropdown'),
#             dcc.Dropdown(
#                 options=[
#                     {'label': 'New York City', 'value': 'NYC'},
#                     {'label': u'Montréal', 'value': 'MTL'},
#                     {'label': 'San Francisco', 'value': 'SF'}
#                 ],
#                 value=['MTL', 'SF'],
#                 multi=True
#                 ),

#             html.Label('Radio Items'),
#             dcc.RadioItems(
#                 options=[
#                     {'label': 'New York City', 'value': 'NYC'},
#                     {'label': u'Montréal', 'value': 'MTL'},
#                     {'label': 'San Francisco', 'value': 'SF'}
#                 ],
#                 value='MTL'
#                 ),

# @app.callback(Output('output-state', 'children'),
#               [Input('submit-button-state', 'n_clicks')],
#               [State('input-1-state', 'value'),
#                State('input-2-state', 'value')])
# def update_output(n_clicks, input1, input2):
#     return u'''
#         The Button has been pressed {} times,
#         Input 1 is "{}",
#         and Input 2 is "{}"
#     '''.format(n_clicks, input1, input2)
    
    
# @app.server.route('{}<image_path>.png'.format(node_static_image_route))
# def serve_node_image(image_path):
#     image_name = '{}.png'.format(image_path)
#     if image_name not in list_of_node_images:
#         raise Exception('"{}" is excluded from the allowed static files'.format(image_path))
#     return flask.send_from_directory(node_image_directory, image_name)


/Users/chrishamblin/miniconda3/envs/graph_viz/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/Users/chrishamblin/miniconda3/envs/graph_viz/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [326]:
app.run_server()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [26/Jun/2020 15:18:27] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Jun/2020 15:18:27] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [26/Jun/2020 15:18:27] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [26/Jun/2020 15:18:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Jun/2020 15:18:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Jun/2020 15:18:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Jun/2020 15:18:30] "POST /_dash-update-component HTTP/1.1" 200 -
/Users/chrishamblin/miniconda3/envs/graph_viz/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/Users/chrishamblin/miniconda3/envs/graph_viz/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

127.0.0.1 - - [26/Jun/2020 15:18:30] "POST

UPDATING STOOOOOOOORERE
UPDATING STOOOOOOOORERE
UPDATING STOOOOOOOORERE

UPDATING FIGUUUUURE
UPDATING FIGUUUUURE
UPDATING FIGUUUUURE



127.0.0.1 - - [26/Jun/2020 15:18:31] "POST /_dash-update-component HTTP/1.1" 200 -


In [213]:
state


{'edge_positions': {0: {'X': [[-1,
     -0.9375,
     -0.875,
     -0.8125,
     -0.75,
     -0.6875,
     -0.625,
     -0.5625,
     -0.5,
     -0.4375,
     -0.375,
     -0.3125,
     -0.25,
     -0.1875,
     -0.125,
     -0.0625,
     0],
    [-1,
     -0.9375,
     -0.875,
     -0.8125,
     -0.75,
     -0.6875,
     -0.625,
     -0.5625,
     -0.5,
     -0.4375,
     -0.375,
     -0.3125,
     -0.25,
     -0.1875,
     -0.125,
     -0.0625,
     0],
    [-1,
     -0.9375,
     -0.875,
     -0.8125,
     -0.75,
     -0.6875,
     -0.625,
     -0.5625,
     -0.5,
     -0.4375,
     -0.375,
     -0.3125,
     -0.25,
     -0.1875,
     -0.125,
     -0.0625,
     0],
    [-1,
     -0.9375,
     -0.875,
     -0.8125,
     -0.75,
     -0.6875,
     -0.625,
     -0.5625,
     -0.5,
     -0.4375,
     -0.375,
     -0.3125,
     -0.25,
     -0.1875,
     -0.125,
     -0.0625,
     0],
    [-1,
     -0.9375,
     -0.875,
     -0.8125,
     -0.75,
     -0.6875,
     -0.625,
     -0.5625,
   

In [216]:
image_name = 'airplane_0001.png'
layer, within_id = nodeid_2_perlayerid(149)
#print(activations[image_name.split('_')[0]][image_name.split('_')[1]])

[array([[[ 1.2824656 ,  0.8565904 ,  0.8818083 , ...,  0.87015975,
          0.855741  ,  0.7828878 ],
        [ 1.2424941 ,  0.23603298,  0.2525151 , ...,  0.23459584,
          0.22942215,  0.64034116],
        [ 1.301531  ,  0.2654904 ,  0.2696002 , ...,  0.24644026,
          0.2499801 ,  0.66129947],
        ...,
        [ 1.521893  ,  0.34898174,  0.3139554 , ...,  0.30267176,
          0.29700178,  0.8708159 ],
        [ 1.4859303 ,  0.3290515 ,  0.44562238, ...,  0.2992803 ,
          0.2746499 ,  0.86019945],
        [ 1.7619598 ,  0.46666557,  0.54549456, ...,  0.6023208 ,
          0.5483697 ,  0.691553  ]],

       [[-0.03734329,  0.17756009,  0.1765095 , ...,  0.18297783,
          0.17204088,  0.16133627],
        [-0.37692112, -0.08926529, -0.08464667, ..., -0.09308489,
         -0.09154018,  0.12175873],
        [-0.4008486 , -0.11605299, -0.10956731, ..., -0.07384205,
         -0.07937238,  0.12824655],
        ...,
        [-0.29320723,  0.17471272,  0.42962372, ..., 

/Users/chrishamblin/miniconda3/envs/graph_viz/lib/python3.7/site-packages/ipykernel_launcher.py:22: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/Users/chrishamblin/miniconda3/envs/graph_viz/lib/python3.7/site-packages/ipykernel_launcher.py:23: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [295]:
print(edge_weights)

{1: [0.13393096771897106, 0.17065674557004942, 0.22843438109956082, 0.21554536803804325, 0.21937029231531824, 0.21865732496510404, 0.1143273308230981, 0.11066470293216926, 0.3083848996134648, 0.11906178153320646, 0.12250828264472524, 0.2549085819787109, 0.10126440253318592], 2: [0.10364892214199276, 0.11814751072434504, 0.2888343040092387, 0.12981509666264301, 0.10236099485570092, 0.1261464246642916, 0.13022122981671022, 0.12924746744562124, 0.22363833715187909, 0.2201996633475538, 0.17473984297618372, 0.12294931867272751, 0.1355681624089815, 0.13662098730953784, 0.10326656722572025, 0.11225775284586524, 0.11832326072405053, 0.11564111721386361, 0.11151365338039465, 0.11474165757599675], 3: [0.12264937774880827, 0.1127526765768474, 0.24119884152364257, 0.10840558234491304, 0.10534195927919754, 0.10874473474096023, 0.1079315683249016, 0.18675519871622193, 0.18388364181848438, 0.14592119810163062, 0.10267213007023734, 0.11320983438133947, 0.11408902408566667]}


In [ ]:
#Hide zoom, pan thingy

dcc.Graph(
    id='my-graph',
    figure={'data': [{'x': [1, 2, 3]}]},
    config={
        'displayModeBar': False
    }
)

In [119]:
#image to numpy array
from PIL import Image
import numpy
im = Image.open("edge_images/0164.png").convert('LA')
np_im = numpy.array(im)
np_im = np_im[:,:,0]
np_im = np.flip(np_im,0)

#numpy array to plotly heatmap
fig = go.Figure(data=go.Heatmap(
                    z = activations['airplane']['0001.png'][3][0]),
                    #z=np_im),
                layout=dict(
                    height=300,
                    width=300))


#fig['layout'].update(scene=dict(aspectmode="data"))
fig.show()
#im.show()

In [253]:
#image to numpy array
from PIL import Image
import numpy
im = Image.open("input_images_testing/airplane_0002.png")
im = im.resize((320,320),resample=Image.NEAREST)
im.show()


# np_im = numpy.array(im)
# np_im.shape


# new_im = Image.fromarray(np_im)
# new_im.save("numpy_altered_sample2.png")


In [195]:
import plotly.graph_objects as go



show = [True,False,False,False,True,False,False,False]

widths = [.01,.1,.5,.8,1,2,3,10] 


layer_colors = ['rgba(31,119,180,', 
                'rgba(255,127,14,',
                'rgba(44,160,44,', 
                'rgba(214,39,40,',
                'rgba(39, 208, 214,', 
                'rgba(242, 250, 17,',
                'rgba(196, 94, 255,',
                'rgba(193, 245, 5,',
                'rgba(245, 85, 5,',
                'rgba(5, 165, 245,',
                'rgba(245, 5, 105,',
                'rgba(218, 232, 23,',
                'rgba(148, 23, 232,',
                'rgba(23, 232, 166,',]


edges_select_df = get_thresholded_edges(threshold = .1)


widths = []
for row in edges_select_df.itertuples():
    widths.append(row.rank_score)
    


fig2 = go.Figure()

# for i in range(4):
#     fig2.add_trace(go.Scatter(
#         x=[1+i, 2+i, 3+i],
#         y=[2+i, 2+i, 2+i],
#         legendgroup="group1",
#         name="group1",
#         mode="lines",
#         line=dict(color=layer_colors[i]+'1)',width = widths[i]),
#         text=[None, "Text B", None],
#         hoverinfo = 'text',
#         showlegend= show[i]
#     ))

# for i in range(4,8):
#     fig2.add_trace(go.Scatter(
#         x=[1+i, 2+i, 3+i],
#         y=[4+i, 9+i, 2+i],
#         legendgroup="group2",
#         name="group2",
#         mode="lines",
#         line=dict(color=layer_colors[i]+'1)',width = widths[i]),
#         showlegend = show[i]
#     ))


# fig3 = go.Figure()

# for i in range(len(layer_colors)):
#     fig3.add_trace(go.Scatter(
#         x=[1, 2, 3],
#         y=[i, i, i],
#         mode="lines",
#         line=dict(color=layer_colors[i]+'1)'),

#     ))    
  
fig4 = go.Figure()

def edge_width_scaling(x):
    return max(.5,(x*5)**1.5)

def edge_color_scaling(x):
    return max(.4,-(x-1)**4+1)

for i in range(len(widths)):
    fig4.add_trace(go.Scatter(
        x=[1, 2, 3],
        y=[.1*i, .1*i, .1*i],
        mode="lines",
        line=dict(color='rgba(255,0,0,%s)'%edge_color_scaling(widths[i]),width=edge_width_scaling(widths[i])),
        text = widths[i],
        hoverinfo = 'text'
        

    ))    

    
#fig2.show()
fig4.show()
edge_color_scaling(.4)

0.8704000000000001

In [86]:
# from PIL import Image
# im_names = os.listdir('input_images_testing')

# for name in im_names:
#     im = Image.open("input_images_testing/%s"%name)
#     im = im.resize((320,320),resample=Image.NEAREST)
#     im.save("input_images/%s"%name,"PNG")

for i in range(5,9):
    print(i)

5
6
7
8


In [91]:
from PIL import Image

def discrete_colorscale(bvals, colors):
    """
    bvals - list of values bounding intervals/ranges of interest
    colors - list of rgb or hex colorcodes for values in [bvals[k], bvals[k+1]],0<=k < len(bvals)-1
    returns the plotly  discrete colorscale
    """
    if len(bvals) != len(colors)+1:
        raise ValueError('len(boundary values) should be equal to  len(colors)+1')
    bvals = sorted(bvals)     
    nvals = [(v-bvals[0])/(bvals[-1]-bvals[0]) for v in bvals]  #normalized values
    
    dcolorscale = [] #discrete colorscale
    for k in range(len(colors)):
        dcolorscale.extend([[nvals[k], colors[k]], [nvals[k+1], colors[k]]])
    return dcolorscale    
bvals = [2, 15, 40, 65, 90]

colors = ['#09ffff', '#19d3f3', '#e763fa' , '#ab63fa']

dcolorsc = discrete_colorscale(bvals, colors)

bvals = np.array(bvals)
tickvals = [np.mean(bvals[k:k+2]) for k in range(len(bvals)-1)] #position with respect to bvals where ticktext is displayed
ticktext = [f'<{bvals[1]}'] + [f'{bvals[k]}-{bvals[k+1]}' for k in range(1, len(bvals)-2)]+[f'>{bvals[-2]}']

z = np.random.randint(bvals[0],  bvals[-1]+1, size=(20, 20))
heatmap = go.Heatmap(z=z, 
                     colorscale = dcolorsc, 
                     colorbar = dict(thickness=25, 
                                     tickvals=tickvals, 
                                     ticktext=ticktext))
fig = go.Figure(data=[heatmap])
fig.update_layout(width=500, height=500)

#fig.show()
#im.show()
print(dcolorsc)



from PIL import Image

def rgb2hex(r, g, b):
    return '#{:02x}{:02x}{:02x}'.format(r, g, b)


def image2heatmap(image_path,resize = False,size = (32,32)):          #displays image as a plotly heatmap object, with colors preserved 
    img = Image.open(image_path)
    if resize:
        img = img.resize(size,resample=Image.NEAREST)
    pixels = img.convert('RGBA').load() #rgba values
    width, height = img.size   #width and height of image
    num_pixels = width*height
    step = 1/num_pixels
    
    colorscale = []           
    z = []
    
    i = 0
    for y in range(height):
        z.append([])
        for x in range(width):
            z[-1].append(step*(i+.5))
            r, g, b, a = pixels[x, y]
            #colorscale.append([step*i,'rgba(%s,%s,%s,%s)'%(r,b,g,a)])
            #colorscale.append([step*(i+1),'rgba(%s,%s,%s,%s)'%(r,b,g,a)])
            colorscale.append([step*i,rgb2hex(r, g, b)])
            colorscale.append([step*(i+1),rgb2hex(r, g, b)])
            i+=1
    print(colorscale)      
    heatmap = go.Heatmap(z=np.flip(np.array(z),0), 
                     colorscale = colorscale, 
                     showscale = False)
    fig = go.Figure(data=[heatmap])

    fig.update_layout(width=500, height=500)
    return fig

img = Image.open("input_images_testing/airplane_0003.png")
img.show()
fig = image2heatmap("input_images_testing/airplane_0003.png",resize=True)
fig.show()

[[0.0, '#09ffff'], [0.14772727272727273, '#09ffff'], [0.14772727272727273, '#19d3f3'], [0.4318181818181818, '#19d3f3'], [0.4318181818181818, '#e763fa'], [0.7159090909090909, '#e763fa'], [0.7159090909090909, '#ab63fa'], [1.0, '#ab63fa']]
[[0.0, '#fbf9fa'], [0.0009765625, '#fbf9fa'], [0.0009765625, '#f7f5f7'], [0.001953125, '#f7f5f7'], [0.001953125, '#f7f5f7'], [0.0029296875, '#f7f5f7'], [0.0029296875, '#f8f6f7'], [0.00390625, '#f8f6f7'], [0.00390625, '#f9f7f8'], [0.0048828125, '#f9f7f8'], [0.0048828125, '#f8f6f8'], [0.005859375, '#f8f6f8'], [0.005859375, '#f8f6f7'], [0.0068359375, '#f8f6f7'], [0.0068359375, '#f8f6f8'], [0.0078125, '#f8f6f8'], [0.0078125, '#f9f7fa'], [0.0087890625, '#f9f7fa'], [0.0087890625, '#f9f6f9'], [0.009765625, '#f9f6f9'], [0.009765625, '#f8f6f9'], [0.0107421875, '#f8f6f9'], [0.0107421875, '#f9f7f9'], [0.01171875, '#f9f7f9'], [0.01171875, '#faf7fa'], [0.0126953125, '#faf7fa'], [0.0126953125, '#f9f7fa'], [0.013671875, '#f9f7fa'], [0.013671875, '#f9f7f9'], [0.0146484